In [15]:
# costMatrix = [[0,9,14,23,32,50,21,49,30,27,35,28,18],   
# [9,0,21,22,36,52,24,51,36,37,41,30,20],    
# [14,21,0,25,38,5,31,7,36,43,29,7,6],    
# [23,22,25,0,42,12,35,17,44,31,31,11,6],
# [32,36,38,42,0,22,37,16,46,37,29,13,14],   
# [50,52,5,12,22,0,41,23,10,39,9,17,16],   
# [21,24,31,35,37,41,0,26,21,19,10,25,12],  
# [49,51,7,17,16,23,26,0,30,28,16,27,12],   
# [30,36,36,44,46,10,21,30,0,25,22,10,20],    
# [27,37,43,31,37,39,19,28,25,0,20,16,8],   
# [35,41,29,31,29,9,10,16,22,20,0,10,10],   
# [28,30,7,11,13,17,25,27,10,16,10,0,10],
# [18,20, 6, 6,14,16,12,12,20,8, 10,10,0]]

# len(costMatrix)

def read_vrpspd_file(file_path):
    with open(file_path, "r") as f:
        lines = [line.strip() for line in f.readlines() if line.strip() and "~" not in line]

    cost_matrix = []
    delivery = []
    pickup = []
    vehicle_capacities = []

    i = 0

    # 1. Read cost matrix
    if "Cost matrix" in lines[i]:
        i += 1

    # đọc từng dòng số -> ma trận
    while i < len(lines) and "Delivery quantities" not in lines[i]:
        row = list(map(float, lines[i].split()))
        cost_matrix.append(row)
        i += 1

    # 2. Read delivery
    if "Delivery quantities" in lines[i]:
        i += 1

    delivery = list(map(int, map(float, lines[i].split())))
    i += 1

    # 3. Read pickup
    if "Pick-up quantities" in lines[i]:
        i += 1

    pickup = list(map(int, map(float, lines[i].split())))
    i += 1

    # 4. Read vehicle capacity (LIST of capacities)
    if "Vehicle capacity" in lines[i]:
        i += 1
        vehicle_capacities = list(map(int, lines[i].split()))
        i += 1

    # === ADD DEPOT ===
    delivery = [0] + delivery
    pickup = [0] + pickup

    # === FINAL VARIABLES ===
    numberOfVehicles = len(vehicle_capacities)

    # if all vehicles have same capacity -> use that as capacityOfVehicle
    if len(set(vehicle_capacities)) == 1:
        capacityOfVehicle = vehicle_capacities[0]
    else:
        # different capacity vehicles
        capacityOfVehicle = max(vehicle_capacities)

    return cost_matrix, delivery, pickup, vehicle_capacities, capacityOfVehicle, numberOfVehicles


# ===== USAGE =====
file_path = "D:\\VRP\\VRPSPD\\class4\\R101_40_08.txt"   # <- thay đường dẫn file thật của bạn
costMatrix, demand, pickup, vehicle_caps, capacityOfVehicle, numberOfVehicles = read_vrpspd_file(file_path)

print("\n✅ costMatrix =", costMatrix)
print("\n✅ demand =", demand)
print("\n✅ pickup =", pickup)
print("\n✅ vehicle_capacities =", vehicle_caps)
print("✅ capacityOfVehicle =", capacityOfVehicle)
print("✅ numberOfVehicles =", numberOfVehicles)



✅ costMatrix = [[0.0, 16.0, 18.0, 23.0, 25.0, 21.0, 12.0, 22.0, 27.0, 33.0, 26.0, 34.0, 15.0, 12.0, 33.0, 31.0, 30.0, 31.0, 16.0, 33.0, 32.0, 19.0, 27.0, 37.0, 30.0, 34.0, 12.0, 5.0, 7.0, 30.0, 26.0, 18.0, 34.0, 25.0, 37.0, 42.0, 42.0, 22.0, 43.0, 34.0, 12.0], [16.0, 0.0, 33.0, 15.0, 33.0, 33.0, 25.0, 22.0, 32.0, 18.0, 16.0, 27.0, 17.0, 27.0, 47.0, 46.0, 43.0, 41.0, 23.0, 29.0, 17.0, 30.0, 40.0, 47.0, 28.0, 38.0, 20.0, 11.0, 12.0, 25.0, 12.0, 11.0, 21.0, 13.0, 25.0, 28.0, 41.0, 36.0, 57.0, 42.0, 25.0], [18.0, 33.0, 0.0, 35.0, 21.0, 24.0, 17.0, 37.0, 37.0, 48.0, 44.0, 51.0, 24.0, 10.0, 22.0, 13.0, 26.0, 33.0, 28.0, 48.0, 50.0, 11.0, 13.0, 24.0, 35.0, 31.0, 17.0, 23.0, 21.0, 39.0, 44.0, 36.0, 52.0, 40.0, 49.0, 56.0, 55.0, 16.0, 33.0, 26.0, 10.0], [23.0, 15.0, 35.0, 0.0, 25.0, 43.0, 34.0, 36.0, 46.0, 15.0, 30.0, 41.0, 12.0, 33.0, 54.0, 48.0, 52.0, 53.0, 36.0, 43.0, 23.0, 27.0, 37.0, 40.0, 15.0, 27.0, 19.0, 21.0, 17.0, 10.0, 22.0, 25.0, 32.0, 8.0, 15.0, 22.0, 56.0, 44.0, 65.0, 34.0, 25.0]

In [16]:
import random
import math
import time
import copy

# ==============================================================================
# CLASS SALS_VND_VRPSPD
# Implement toàn bộ thuật toán dựa trên luận văn của Mustafa Avci (2014)
# ==============================================================================

class SALS_VND_VRPSPD:
    def __init__(self, cost_matrix, demand, pickup, capacity, num_vehicles):
        self.cost_matrix = cost_matrix
        self.demand = demand
        self.pickup = pickup
        self.capacity = capacity
        self.num_vehicles = num_vehicles
        self.num_customers = len(cost_matrix) - 1 # Trừ depot
        
        # Danh sách khách hàng (1..n)
        self.customers_list = list(range(1, len(cost_matrix)))

    # --------------------------------------------------------------------------
    # 1. CÁC HÀM PHỤ TRỢ (Cost & Feasibility)
    # --------------------------------------------------------------------------
    
    def calculate_route_cost(self, route):
        """Tính chi phí của một tuyến đơn lẻ."""
        if not route:
            return 0
        cost = 0
        # Từ depot đến điểm đầu
        cost += self.cost_matrix[0][route[0]]
        # Các điểm giữa
        for i in range(len(route) - 1):
            cost += self.cost_matrix[route[i]][route[i+1]]
        # Từ điểm cuối về depot
        cost += self.cost_matrix[route[-1]][0]
        return cost

    def calculate_total_cost(self, routes):
        """Tính tổng chi phí của toàn bộ giải pháp."""
        return sum(self.calculate_route_cost(r) for r in routes)

    def check_route_feasibility(self, route):
        """
        Kiểm tra tính hợp lệ của một tuyến (Mục 4.3.3).
        VRPSPD: Tải trọng thay đổi non-monotonic.
        """
        if not route:
            return True
            
        # Tổng hàng giao và nhận không được vượt quá tải trọng xe
        total_d = sum(self.demand[c] for c in route)
        total_p = sum(self.pickup[c] for c in route)
        if total_d > self.capacity or total_p > self.capacity:
            return False

        # Mô phỏng hành trình: Xe xuất phát với toàn bộ hàng cần giao
        current_load = total_d
        
        # Kiểm tra từng điểm
        for c in route:
            current_load = current_load - self.demand[c] + self.pickup[c]
            if current_load > self.capacity:
                return False # Vi phạm tải trọng giữa đường
        
        return True

    def check_solution_feasibility(self, routes):
        """Kiểm tra toàn bộ giải pháp."""
        # Kiểm tra số lượng xe
        if len(routes) > self.num_vehicles:
            return False
        # Kiểm tra từng tuyến
        for r in routes:
            if not self.check_route_feasibility(r):
                return False
        return True

    # --------------------------------------------------------------------------
    # 2. KHỞI TẠO NGẪU NHIÊN (Mục 4.3.2 - Figure 4.6)
    # --------------------------------------------------------------------------
    
    def generate_initial_solution(self):
        """
        Tạo lời giải ban đầu ngẫu nhiên:
        Lấy ngẫu nhiên khách hàng, thêm vào tuyến hiện tại. Nếu đầy thì tạo tuyến mới.
        """
        unvisited = self.customers_list[:]
        random.shuffle(unvisited)
        
        routes = []
        current_route = []
        
        while unvisited:
            customer = unvisited.pop(0)
            # Thử thêm vào tuyến hiện tại
            trial_route = current_route + [customer]
            
            if self.check_route_feasibility(trial_route):
                current_route.append(customer)
            else:
                # Nếu không thêm được, đóng tuyến cũ, mở tuyến mới
                if current_route:
                    routes.append(current_route)
                current_route = [customer]
        
        if current_route:
            routes.append(current_route)
            
        return routes

    # --------------------------------------------------------------------------
    # 3. NEIGHBORHOOD STRUCTURES (Mục 4.3.4)
    # Implement các hàm sinh hàng xóm. Trả về (New Routes, Cost) hoặc None
    # --------------------------------------------------------------------------

    # --- Intra-route Moves (Trong cùng 1 tuyến) ---
    
    def move_general_swap(self, routes, r_idx):
        """Đổi chỗ 2 vị trí bất kỳ trong tuyến."""
        route = routes[r_idx][:]
        if len(route) < 2: return None
        i, j = random.sample(range(len(route)), 2)
        route[i], route[j] = route[j], route[i]
        
        if self.check_route_feasibility(route):
            new_routes = copy.deepcopy(routes)
            new_routes[r_idx] = route
            return new_routes
        return None

    def move_2opt(self, routes, r_idx):
        """Đảo ngược một đoạn trong tuyến."""
        route = routes[r_idx][:]
        if len(route) < 3: return None
        i = random.randint(0, len(route) - 2)
        j = random.randint(i + 1, len(route) - 1)
        
        new_route = route[:i] + route[i:j+1][::-1] + route[j+1:]
        
        if self.check_route_feasibility(new_route):
            new_routes = copy.deepcopy(routes)
            new_routes[r_idx] = new_route
            return new_routes
        return None

    def move_relocate_intra(self, routes, r_idx):
        """Di chuyển 1 khách hàng đến vị trí khác trong tuyến."""
        route = routes[r_idx][:]
        if len(route) < 2: return None
        i = random.randint(0, len(route) - 1)
        cust = route.pop(i)
        j = random.randint(0, len(route)) # Vị trí chèn mới
        route.insert(j, cust)
        
        if self.check_route_feasibility(route):
            new_routes = copy.deepcopy(routes)
            new_routes[r_idx] = route
            return new_routes
        return None

    def move_block_insertion_intra(self, routes, r_idx):
        """Di chuyển khối 2 khách hàng trong tuyến."""
        route = routes[r_idx][:]
        if len(route) < 4: return None
        i = random.randint(0, len(route) - 2) # Start block
        block = route[i:i+2]
        del route[i:i+2]
        j = random.randint(0, len(route))
        
        new_route = route[:j] + block + route[j:]
        
        if self.check_route_feasibility(new_route):
            new_routes = copy.deepcopy(routes)
            new_routes[r_idx] = new_route
            return new_routes
        return None

    # --- Inter-route Moves (Giữa 2 tuyến) ---

    def move_swap_1_1(self, routes, r1_idx, r2_idx):
        """Swap(1,1): Tráo đổi 1 khách hàng giữa 2 tuyến."""
        r1 = routes[r1_idx][:]
        r2 = routes[r2_idx][:]
        if not r1 or not r2: return None
        
        i = random.randint(0, len(r1)-1)
        j = random.randint(0, len(r2)-1)
        
        r1[i], r2[j] = r2[j], r1[i]
        
        if self.check_route_feasibility(r1) and self.check_route_feasibility(r2):
            new_routes = copy.deepcopy(routes)
            new_routes[r1_idx] = r1
            new_routes[r2_idx] = r2
            return new_routes
        return None

    def move_shift_1_0(self, routes, r1_idx, r2_idx):
        """Shift(1,0): Chuyển 1 khách từ r1 sang r2."""
        r1 = routes[r1_idx][:]
        r2 = routes[r2_idx][:]
        if not r1: return None
        
        i = random.randint(0, len(r1)-1)
        cust = r1.pop(i)
        j = random.randint(0, len(r2))
        r2.insert(j, cust)
        
        if self.check_route_feasibility(r1) and self.check_route_feasibility(r2):
            new_routes = copy.deepcopy(routes)
            new_routes[r1_idx] = r1 if r1 else [] # Xử lý tuyến rỗng sau này
            new_routes[r2_idx] = r2
            # Clean up empty routes
            return [r for r in new_routes if r]
        return None

    def move_shift_2_0(self, routes, r1_idx, r2_idx):
        """Shift(2,0): Chuyển khối 2 khách từ r1 sang r2."""
        r1 = routes[r1_idx][:]
        r2 = routes[r2_idx][:]
        if len(r1) < 2: return None
        
        i = random.randint(0, len(r1)-2)
        block = r1[i:i+2]
        del r1[i:i+2]
        
        j = random.randint(0, len(r2))
        r2 = r2[:j] + block + r2[j:]
        
        if self.check_route_feasibility(r1) and self.check_route_feasibility(r2):
            new_routes = copy.deepcopy(routes)
            new_routes[r1_idx] = r1
            new_routes[r2_idx] = r2
            return [r for r in new_routes if r]
        return None

    def move_swap_2_1(self, routes, r1_idx, r2_idx):
        """Swap(2,1): Tráo khối 2 khách ở r1 với 1 khách ở r2."""
        r1 = routes[r1_idx][:]
        r2 = routes[r2_idx][:]
        if len(r1) < 2 or len(r2) < 1: return None
        
        i = random.randint(0, len(r1)-2) # Block start in r1
        j = random.randint(0, len(r2)-1) # Customer in r2
        
        block = r1[i:i+2]
        cust = r2[j]
        
        # Construct new routes logic
        new_r1 = r1[:i] + [cust] + r1[i+2:]
        new_r2 = r2[:j] + block + r2[j+1:]
        
        if self.check_route_feasibility(new_r1) and self.check_route_feasibility(new_r2):
            new_routes = copy.deepcopy(routes)
            new_routes[r1_idx] = new_r1
            new_routes[r2_idx] = new_r2
            return new_routes
        return None

    def move_swap_2_2(self, routes, r1_idx, r2_idx):
        """Swap(2,2): Tráo khối 2 khách ở r1 với khối 2 khách ở r2."""
        r1 = routes[r1_idx][:]
        r2 = routes[r2_idx][:]
        if len(r1) < 2 or len(r2) < 2: return None
        
        i = random.randint(0, len(r1)-2)
        j = random.randint(0, len(r2)-2)
        
        block1 = r1[i:i+2]
        block2 = r2[j:j+2]
        
        new_r1 = r1[:i] + block2 + r1[i+2:]
        new_r2 = r2[:j] + block1 + r2[j+2:]
        
        if self.check_route_feasibility(new_r1) and self.check_route_feasibility(new_r2):
            new_routes = copy.deepcopy(routes)
            new_routes[r1_idx] = new_r1
            new_routes[r2_idx] = new_r2
            return new_routes
        return None

    def get_random_neighbor(self, routes):
        """
        Chọn ngẫu nhiên 1 trong 9 loại nước đi và thực hiện.
        Thử tối đa 20 lần để tìm được nước đi hợp lệ.
        """
        # Mapping index to function type
        # Intra: 0-3, Inter: 4-8
        for _ in range(20):
            move_type = random.randint(0, 8)
            num_routes = len(routes)
            new_sol = None
            
            # Chọn ngẫu nhiên tuyến
            if move_type <= 3: # Intra moves
                r_idx = random.randint(0, num_routes - 1)
                if move_type == 0: new_sol = self.move_general_swap(routes, r_idx)
                elif move_type == 1: new_sol = self.move_2opt(routes, r_idx)
                elif move_type == 2: new_sol = self.move_relocate_intra(routes, r_idx)
                elif move_type == 3: new_sol = self.move_block_insertion_intra(routes, r_idx)
            else: # Inter moves
                if num_routes < 2: continue
                r1, r2 = random.sample(range(num_routes), 2)
                if move_type == 4: new_sol = self.move_swap_1_1(routes, r1, r2)
                elif move_type == 5: new_sol = self.move_shift_1_0(routes, r1, r2)
                elif move_type == 6: new_sol = self.move_shift_2_0(routes, r1, r2)
                elif move_type == 7: # Swap 2-1 (Random direction)
                    if random.random() < 0.5: new_sol = self.move_swap_2_1(routes, r1, r2)
                    else: new_sol = self.move_swap_2_1(routes, r2, r1)
                elif move_type == 8: new_sol = self.move_swap_2_2(routes, r1, r2)
            
            if new_sol is not None:
                return new_sol
        
        return None # Không tìm được sau nhiều lần thử

    # --------------------------------------------------------------------------
    # 4. VND (Variable Neighborhood Descent) - Mục 4.2 & 4.3.6
    # Ordering 2: Swap(1,1), Shift(1,0), Shift(2,0), Swap(2,1), Swap(2,2)
    # Cơ chế: Inter-route trước -> nếu tốt -> Intra-route (Intensification)
    # --------------------------------------------------------------------------

    def vnd_intra_optimize(self, routes, route_indices):
        """Tối ưu cục bộ (2-opt, Swap) cho các tuyến vừa bị thay đổi."""
        improved = True
        while improved:
            improved = False
            for r_idx in route_indices:
                if r_idx >= len(routes): continue
                # Thử 2-opt (mạnh nhất cho intra)
                # Duyệt tất cả các cặp i, j
                best_route = routes[r_idx]
                best_cost = self.calculate_route_cost(best_route)
                
                for i in range(len(best_route) - 2):
                    for j in range(i + 1, len(best_route) - 1):
                        # Giả lập 2-opt
                        new_r = best_route[:i] + best_route[i:j+1][::-1] + best_route[j+1:]
                        if self.check_route_feasibility(new_r):
                            cost = self.calculate_route_cost(new_r)
                            if cost < best_cost:
                                routes[r_idx] = new_r
                                best_cost = cost
                                improved = True
        return routes

    def run_vnd(self, current_routes):
        """
        Thực thi VND theo Ordering 2 (Inter-route).
        Chiến lược: Best Improvement trong neighborhood (duyệt hết khả năng).
        Do không gian lớn, ta dùng First Improvement với số lần thử giới hạn hoặc duyệt ngẫu nhiên
        để đảm bảo tốc độ, hoặc duyệt toàn bộ nếu số lượng khách nhỏ.
        Ở đây cài đặt First Improvement with systematic search.
        """
        best_routes = copy.deepcopy(current_routes)
        best_cost = self.calculate_total_cost(best_routes)
        
        k = 1
        k_max = 5 # 5 cấu trúc inter-route
        
        while k <= k_max:
            improvement_found = False
            
            # Tạo danh sách các cặp tuyến để duyệt
            num_r = len(best_routes)
            pairs = [(i, j) for i in range(num_r) for j in range(num_r) if i != j]
            
            # Duyệt qua các hàng xóm của cấu trúc k
            for r1, r2 in pairs:
                # Tùy thuộc vào k mà gọi hàm tương ứng
                # Ordering 2: Swap(1,1), Shift(1,0), Shift(2,0), Swap(2,1), Swap(2,2)
                
                # Cần duyệt qua CÁC PHẦN TỬ trong r1, r2 để thực hiện move
                # Để code gọn, ta dùng lại các hàm move_... nhưng cần sửa lại để nhận index cụ thể
                # Tuy nhiên, các hàm move_... ở trên là random. 
                # -> Trong VND chuẩn, phải duyệt toàn bộ.
                # -> Để đơn giản hóa nhưng vẫn hiệu quả, ta sẽ lặp thử nghiệm nhiều lần random (Stochastic VND)
                # hoặc viết hàm duyệt chi tiết. Ở đây dùng Stochastic VND (thử nhiều lần) cho đơn giản code.
                
                for _ in range(20): # Thử 20 lần random trên cặp tuyến này
                    candidate = None
                    if k == 1: candidate = self.move_swap_1_1(best_routes, r1, r2)
                    elif k == 2: candidate = self.move_shift_1_0(best_routes, r1, r2)
                    elif k == 3: candidate = self.move_shift_2_0(best_routes, r1, r2)
                    elif k == 4: candidate = self.move_swap_2_1(best_routes, r1, r2)
                    elif k == 5: candidate = self.move_swap_2_2(best_routes, r1, r2)
                    
                    if candidate:
                        cost = self.calculate_total_cost(candidate)
                        if cost < best_cost:
                            # Nếu cải thiện -> Intra optimization
                            candidate = self.vnd_intra_optimize(candidate, [r1, r2])
                            final_cost = self.calculate_total_cost(candidate)
                            
                            if final_cost < best_cost:
                                best_routes = candidate
                                best_cost = final_cost
                                improvement_found = True
                                break # First improvement
                if improvement_found: break
            
            if improvement_found:
                k = 1 # Quay lại cấu trúc đầu tiên
            else:
                k += 1 # Chuyển sang cấu trúc tiếp theo
                
        return best_routes, best_cost

    # --------------------------------------------------------------------------
    # 5. SALS (Giai đoạn 1) - Mục 4.1 & 4.3.5
    # --------------------------------------------------------------------------

    def run_sals_stage1(self, max_iter_no_improve=500):
        print("--- Bắt đầu Giai đoạn 1: SALS + VND ---")
        
        # 1. Khởi tạo ngẫu nhiên
        current_sol = self.generate_initial_solution()
        current_cost = self.calculate_total_cost(current_sol)
        
        best_sol = copy.deepcopy(current_sol)
        best_cost = current_cost
        
        # Local Best (cho SALS)
        local_best_sol = copy.deepcopy(current_sol)
        local_best_cost = current_cost
        
        # Tham số SALS
        theta = 0 # Threshold parameter (gọi là t trong bài báo)
        # Khởi tạo theta (t) dựa trên công thức 4.3: t = 1 + a1*a2
        # Ban đầu giả sử a1=0, a2=0 -> t=1 (chỉ chấp nhận tốt hơn hoặc bằng)
        # Nhưng để đa dạng hóa ban đầu, ta cho phép t > 1
        t = 1.05 
        
        iter_count = 0
        no_improve_count = 0
        Ci = 0 # Số lượng lời giải cải thiện
        
        while no_improve_count < max_iter_no_improve:
            iter_count += 1
            
            # Sinh hàng xóm ngẫu nhiên
            neighbor = self.get_random_neighbor(current_sol)
            if neighbor is None: continue
            
            neighbor_sol = neighbor
            neighbor_cost = self.calculate_total_cost(neighbor_sol)
            
            # Điều kiện chấp nhận: f(x') <= t * f(xlb)
            # Lưu ý: Trong code của bạn trước đây dùng current, nhưng bài báo dùng local_best (xlb)
            # Logic: if f(x') <= t * f(current) (Theo SALS gốc của Alabas-Uslu)
            # Tuy nhiên Avci (4.3.5) dùng t * f(current) trong hình 4.12
            
            threshold_val = t * current_cost
            
            accepted = False
            if neighbor_cost <= threshold_val:
                current_sol = neighbor_sol
                current_cost = neighbor_cost
                accepted = True
                
                # Nếu tốt hơn Local Best -> Trigger VND
                if current_cost < local_best_cost:
                    # Cải tiến bằng VND
                    refined_sol, refined_cost = self.run_vnd(current_sol)
                    
                    # Update Local Best
                    if refined_cost < local_best_cost:
                        local_best_sol = refined_sol
                        local_best_cost = refined_cost
                        current_sol = refined_sol # Di chuyển luôn sang điểm tốt này
                        current_cost = refined_cost
                        
                        no_improve_count = 0 # Reset đếm dừng
                        Ci += 1
                        
                        # Update Global Best
                        if local_best_cost < best_cost:
                            best_sol = copy.deepcopy(local_best_sol)
                            best_cost = local_best_cost
                            print(f"[SALS] New Best found: {best_cost:.2f} (Iter {iter_count})")
            
            if not accepted:
                no_improve_count += 1
            
            # Cập nhật tham số t (Adaptive)
            # a1 = f(best)/f(current), a2 = Ci / i
            a1 = best_cost / current_cost if current_cost > 0 else 1
            a2 = Ci / iter_count if iter_count > 0 else 0
            t = 1 + a1 * a2
            
        print(f"Giai đoạn 1 kết thúc. Best Cost: {best_cost:.2f}")
        return best_sol, best_cost

    # --------------------------------------------------------------------------
    # 6. PERTURBATION & STAGE 2 (Mục 4.3.7)
    # --------------------------------------------------------------------------

    def apply_perturbation(self, routes, pert_length):
        """Thực hiện một chuỗi các nước đi ngẫu nhiên để thoát local optima."""
        current_routes = copy.deepcopy(routes)
        
        # Chỉ dùng Inter-route moves cho perturbation (như bài báo nói)
        # Các move type: 4, 5, 6, 7, 8
        moves_done = 0
        gamma = 1.0 # Acceptance param cho perturbation
        
        while moves_done < pert_length:
            # Chọn ngẫu nhiên 2 tuyến
            if len(current_routes) < 2: break
            r1, r2 = random.sample(range(len(current_routes)), 2)
            move_type = random.randint(4, 8)
            
            new_sol = None
            if move_type == 4: new_sol = self.move_swap_1_1(current_routes, r1, r2)
            elif move_type == 5: new_sol = self.move_shift_1_0(current_routes, r1, r2)
            elif move_type == 6: new_sol = self.move_shift_2_0(current_routes, r1, r2)
            elif move_type == 7: new_sol = self.move_swap_2_1(current_routes, r1, r2)
            elif move_type == 8: new_sol = self.move_swap_2_2(current_routes, r1, r2)
            
            if new_sol:
                new_cost = self.calculate_total_cost(new_sol)
                current_c = self.calculate_total_cost(current_routes)
                
                # Perturbation acceptance: f(x') <= gamma * f(x)
                if new_cost <= gamma * current_c:
                    current_routes = new_sol
                    moves_done += 1
                else:
                    gamma += 0.01 # Nới lỏng nếu bị kẹt
                    
        return current_routes

    def run_stage2_perturbation(self, initial_sol, max_iter=15):
        print("--- Bắt đầu Giai đoạn 2: VND + Perturbation ---")
        
        current_sol = copy.deepcopy(initial_sol)
        current_cost = self.calculate_total_cost(current_sol)
        
        best_sol = copy.deepcopy(current_sol)
        best_cost = current_cost
        
        m2 = 0 # Số lần VND không cải thiện
        pert_length = 1
        
        while m2 < max_iter:
            # 1. Perturbation
            perturbed_sol = self.apply_perturbation(current_sol, pert_length)
            
            # 2. VND
            refined_sol, refined_cost = self.run_vnd(perturbed_sol)
            
            # 3. Acceptance decision
            if refined_cost < best_cost:
                print(f"[Stage 2] Improvement: {best_cost:.2f} -> {refined_cost:.2f}")
                best_sol = refined_sol
                best_cost = refined_cost
                current_sol = refined_sol
                
                m2 = 0
                pert_length = 1 # Reset độ nhiễu
            else:
                m2 += 1
                if refined_cost == best_cost:
                    pert_length += 1 # Tăng độ nhiễu nếu bị kẹt lại điểm cũ
                # current_sol giữ nguyên là best_sol (để perturbation từ điểm tốt nhất)
        
        return best_sol, best_cost

    # --------------------------------------------------------------------------
    # MAIN SOLVE FUNCTION
    # --------------------------------------------------------------------------
    def solve(self):
        # Giai đoạn 1: SALS + VND
        s1_sol, s1_cost = self.run_sals_stage1(max_iter_no_improve=10000)
        
        # Giai đoạn 2: Perturbation + VND
        final_sol, final_cost = self.run_stage2_perturbation(s1_sol, max_iter=1000)
        
        print("\n=== KẾT QUẢ CUỐI CÙNG ===")
        print(f"Chi phí: {final_cost:.2f}")
        return final_sol, final_cost

# ==============================================================================
# CHẠY THỬ NGHIỆM
# ==============================================================================

# Giả sử các biến costMatrix, demand, pickup... đã có từ cell đọc file của bạn
# Khởi tạo solver
solver = SALS_VND_VRPSPD(costMatrix, demand, pickup, capacityOfVehicle, numberOfVehicles)

# Chạy thuật toán
final_routes, final_cost = solver.solve()

# In chi tiết tuyến đường
print("\nChi tiết tuyến đường:")
for i, r in enumerate(final_routes):
    print(f"Xe {i+1}: 0 -> {' -> '.join(map(str, r))} -> 0  (Load: {sum(demand[c] for c in r)})")

--- Bắt đầu Giai đoạn 1: SALS + VND ---
[SALS] New Best found: 790.00 (Iter 1)
[SALS] New Best found: 705.00 (Iter 1007)
[SALS] New Best found: 704.00 (Iter 1065)
[SALS] New Best found: 688.00 (Iter 3899)
[SALS] New Best found: 687.00 (Iter 3917)
[SALS] New Best found: 677.00 (Iter 3984)
[SALS] New Best found: 667.00 (Iter 7794)
[SALS] New Best found: 660.00 (Iter 7817)
[SALS] New Best found: 659.00 (Iter 8099)
Giai đoạn 1 kết thúc. Best Cost: 659.00
--- Bắt đầu Giai đoạn 2: VND + Perturbation ---
[Stage 2] Improvement: 659.00 -> 658.00
[Stage 2] Improvement: 658.00 -> 654.00
[Stage 2] Improvement: 654.00 -> 652.00
[Stage 2] Improvement: 652.00 -> 651.00
[Stage 2] Improvement: 651.00 -> 649.00
[Stage 2] Improvement: 649.00 -> 638.00
[Stage 2] Improvement: 638.00 -> 637.00
[Stage 2] Improvement: 637.00 -> 635.00
[Stage 2] Improvement: 635.00 -> 631.00
[Stage 2] Improvement: 631.00 -> 628.00
[Stage 2] Improvement: 628.00 -> 627.00

=== KẾT QUẢ CUỐI CÙNG ===
Chi phí: 627.00

Chi tiết tuyế